In [1]:
import pandas as pd
from more_sqlalchemy import get_sql_types

#### Delete and start fresh

In [2]:
# On Windows, go delete the db from the folder
!rm ./databases/slide_example.db

#### Make a connection to a new database

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///databases/slide_example.db', echo=False)

#### Read in the table from a csv

In [4]:
table1 = pd.read_csv('./data/Table1.csv')
table1.head()

,D,G
0,5,a
1,3,a
2,4,b
3,2,b
4,3,c


#### Create a SQL type dictionary

In [5]:
sql_types = get_sql_types(table1)
sql_types

{'D': sqlalchemy.sql.sqltypes.Integer, 'G': sqlalchemy.sql.sqltypes.String}

#### Create and execute the SQL CREATE TABLE schema

In [6]:
schema = pd.io.sql.get_schema(table1, # dataframe
                              'Table1', # name in SQL db
                              keys=None, # primary key
                              con=engine, # connection
                              dtype=sql_types # SQL types
)
print(schema)
engine.execute(schema)


CREATE TABLE "Table1" (
	"D" INTEGER, 
	"G" VARCHAR
)




#### Append the data to the new table

In [7]:
table1.to_sql('Table1', 
              con=engine, 
              dtype=sql_types, 
              index=False,
              if_exists='append')

In [8]:
def create_table(df, table_name, con, keys=None):
    sql_types = get_sql_types(df)
    schema = pd.io.sql.get_schema(df,
                                  table_name,
                                  keys=keys,
                                  con=con,
                                  dtype=sql_types)
    con.execute(schema)
    df.to_sql(table_name, 
              con=con, 
              dtype=sql_types, 
              index=False,
              if_exists='append')

In [9]:
t1 = pd.read_csv('./data/T1.csv')
t1.head()

,G1,X
0,a,5
1,a,2
2,b,3
3,b,1


In [10]:
create_table(t1, 'T1', engine)

In [11]:
t2 = pd.read_csv('./data/T2.csv')
t2.head()

,G2,Y
0,a,3
1,a,2
2,b,4
3,b,1
4,c,1


In [12]:
create_table(t2, 'T2', engine)

#### Test the results

In [13]:
%load_ext pyensae

In [14]:
%SQL_connect ./databases/slide_example.db

In [15]:
%SQL_tables

['T1', 'T2', 'Table1']

In [16]:
%SQL_schema Table1

{0: ('D', int), 1: ('G', str)}

In [19]:
%%SQL
SELECT *
FROM Table1

,D,G
0,5,a
1,3,a
2,4,b
3,2,b
4,3,c
5,5,c
6,4,c


In [17]:
%SQL_schema T1

{0: ('G1', str), 1: ('X', int)}

In [20]:
%%SQL
SELECT *
FROM T1

,G1,X
0,a,5
1,a,2
2,b,3
3,b,1


In [18]:
%SQL_schema T2

{0: ('G2', str), 1: ('Y', int)}

In [21]:
%%SQL
SELECT *
FROM T2

,G2,Y
0,a,3
1,a,2
2,b,4
3,b,1
4,c,1
